In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np
import joblib
from hybrid_model import HybridModel 

In [2]:
# Step 1: Load and preprocess the dataset
file_path = "Sleep_health_and_lifestyle_dataset.csv"  # Replace with your dataset path
data = pd.read_csv(file_path)

# Drop unnecessary columns
data = data.drop(columns=["Person ID"])

# Fill missing values
data.fillna(method='ffill', inplace=True)

# Split the dataset into features and target
X = data.drop(columns=["Sleep Disorder"])
y = data["Sleep Disorder"]

In [3]:
# Encode categorical columns in X
label_encoders = {}
categorical_columns = X.select_dtypes(include=["object"]).columns
for column in categorical_columns:
    le = LabelEncoder()
    X[column] = le.fit_transform(X[column])
    label_encoders[column] = dict(zip(le.classes_, le.transform(le.classes_)))

# Encode target column
if y.dtype == "object":
    target_encoder = LabelEncoder()
    y = target_encoder.fit_transform(y)

In [ ]:
# Step 2: Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Train Random Forest model
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

# Step 4: Generate Random Forest probabilities
rf_proba_train = rf.predict_proba(X_train)
rf_proba_test = rf.predict_proba(X_test)

# Combine RF probabilities with original features
X_train_hybrid = np.hstack((X_train, rf_proba_train))
X_test_hybrid = np.hstack((X_test, rf_proba_test))

# Step 5: Standardize the hybrid features
scaler = StandardScaler()
X_train_hybrid = scaler.fit_transform(X_train_hybrid)
X_test_hybrid = scaler.transform(X_test_hybrid)

# Step 6: Train the ANN model
ann = Sequential([
    Dense(128, activation='relu', input_dim=X_train_hybrid.shape[1]),
    Dense(64, activation='relu'),
    Dense(len(set(y)), activation='softmax')  # Adjust output layer for the number of classes
])
ann.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
ann.fit(X_train_hybrid, y_train, epochs=50, batch_size=16, verbose=1)

# Step 7: Create the hybrid model object
hybrid_model = HybridModel(rf_model=rf, ann_model=ann, scaler=scaler)

# Step 8: Save the hybrid model
hybrid_model_path = "rfann_hybrid_model.pkl"
joblib.dump(hybrid_model, hybrid_model_path)
print(f"Hybrid model saved to {hybrid_model_path}")


Epoch 1/50
19/19 [==============================] - 0s 950us/step - loss: 0.7399 - accuracy: 0.7993
Epoch 2/50
19/19 [==============================] - 0s 1ms/step - loss: 0.2911 - accuracy: 0.9632
Epoch 3/50
19/19 [==============================] - 0s 1ms/step - loss: 0.1472 - accuracy: 0.9666
Epoch 4/50
19/19 [==============================] - 0s 2ms/step - loss: 0.0992 - accuracy: 0.9699
Epoch 5/50
19/19 [==============================] - 0s 1ms/step - loss: 0.0867 - accuracy: 0.9732
Epoch 6/50
19/19 [==============================] - 0s 1ms/step - loss: 0.0842 - accuracy: 0.9699
Epoch 7/50
19/19 [==============================] - 0s 2ms/step - loss: 0.0846 - accuracy: 0.9699
Epoch 8/50
19/19 [==============================] - 0s 2ms/step - loss: 0.0749 - accuracy: 0.9732
Epoch 9/50
19/19 [==============================] - 0s 2ms/step - loss: 0.0711 - accuracy: 0.9699
Epoch 10/50
19/19 [==============================] - 0s 1ms/step - loss: 0.0721 - accuracy: 0.9699
Epoch 11/50
19/19

In [6]:
from hybrid_model import HybridModel  # Import from the correct module
import joblib

# Assuming `rf`, `ann`, and `scaler` are your trained models and scaler
hybrid_model = HybridModel(rf_model=rf, ann_model=ann, scaler=scaler)

# Save the model
hybrid_model_path = "rfann_hybrid_model.pkl"
joblib.dump(hybrid_model, hybrid_model_path)
print(f"Hybrid model saved to {hybrid_model_path}")


Hybrid model saved to rfann_hybrid_model.pkl
